<a href="https://colab.research.google.com/github/redpineK/deeplearning/blob/master/LIME_for_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((-1,28,28,1)).astype('float32') / 255.0
x_test = x_test.reshape((-1,28,28,1)).astype('float32') / 255.0

In [ ]:
import numpy as np
def to_rgb(x):
    x_rgb = np.zeros((x.shape[0], 28, 28, 3))
    for i in range(3):
        x_rgb[..., i] = x[..., 0]
    return x_rgb
x_train = to_rgb(x_train)
x_test = to_rgb(x_test)

In [ ]:
print(keras.__version__)

model = keras.Sequential(
    [
     keras.Input(shape=(28,28,3)),
     layers.Conv2D(16, 3, activation='relu'),
     layers.MaxPooling2D(),
     layers.Flatten(),
     layers.Dense(10)
    ]
)

In [ ]:
model.compile(
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=keras.optimizers.Adam(),
  metrics=['accuracy']
)

In [ ]:
# epochㄴ=2 로 짧게 훈련
#
model.fit(
        x_train, 
        y_train, 
        epochs=10, 
        batch_size=32, 
        validation_data = (x_test, y_test))

LIME (Local Interpretable Model-agnostic Explanation)

In [ ]:
# LIME 패키지 설치
#
!pip install lime

In [ ]:
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import random

In [ ]:
# 대상 이미지 
#
idx_test = 64
print("y_test[", str(idx_test), "] == ", y_test[idx_test])
# idx_train = 10
explainer = lime_image.LimeImageExplainer(random_state=42)
explanation = explainer.explain_instance(
         x_test[idx_test], 
         model.predict
)


In [ ]:
# 판단에 참조된 부분
#
image, mask = explanation.get_image_and_mask(
           # model.predict(
           #      x_test[idx_test].reshape((1,28,28,3))
           # ).argmax(axis=1)[0],
            y_test[idx_test],
            # 8,
         positive_only=True,
         hide_rest=True)
print("mask == ", mask)
plt.imshow(mark_boundaries(image, mask))
plt.show()